# Загрузка данных

In [40]:
import json

# Чтение данных из файлов
def load_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return [json.loads(line) for line in f]

train_data = load_data('ml_trainings.alien_translation/train')  # Путь к файлу train
val_data = load_data('ml_trainings.alien_translation/val')  # Путь к файлу val
test_data = load_data('ml_trainings.alien_translation/test_no_reference')  # Путь к файлу test_no_reference

# Пример структуры данных
print(train_data[0])  # Пример одного элемента из train_data


{'dst': '- Intriguing.', 'src': '◄▴◓◠▨ ◨▽◠▦◈◬◓▪▼◬▵'}


# Токенизация

In [14]:
!pip install datasets transformers spacy


In [15]:
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 952.9 kB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [41]:
import spacy
from collections import Counter

# Загрузка модели для токенизации английского
spacy_en = spacy.load('en_core_web_sm')

# Функция для токенизации
def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

# Для вымышленного языка будем использовать токенизацию по символам
def tokenize_conlang(text):
    return list(text)  # Разбиение по символам для вымышленного языка

# Токенизация данных
train_src = [item['src'] for item in train_data]
train_dst = [item['dst'] for item in train_data]

# Применение токенизации
train_src_tokens = [tokenize_conlang(src) for src in train_src]  # Вымышленный язык
train_dst_tokens = [tokenize_en(dst) for dst in train_dst]  # Английский язык

# Пример
print(train_src_tokens[0])  # Пример токенизации исходного текста
print(train_dst_tokens[0])  # Пример токенизации целевого текста


['◄', '▴', '◓', '◠', '▨', ' ', '◨', '▽', '◠', '▦', '◈', '◬', '◓', '▪', '▼', '◬', '▵']
['-', 'Intriguing', '.']


In [44]:
from collections import Counter

# Функция для построения словаря
def build_vocab(tokens):
    counter = Counter()
    for sentence in tokens:
        counter.update(sentence)
    vocab = {word: idx for idx, (word, _) in enumerate(counter.items())}
    return vocab

# Построение словарей
src_vocab = build_vocab(train_src_tokens)
dst_vocab = build_vocab(train_dst_tokens)

# Добавление специальных токенов
src_vocab['<unk>'] = len(src_vocab)
src_vocab['<pad>'] = len(src_vocab) + 1
src_vocab['<sos>'] = len(src_vocab) + 2
src_vocab['<eos>'] = len(src_vocab) + 3

dst_vocab['<unk>'] = len(dst_vocab)
dst_vocab['<pad>'] = len(dst_vocab) + 1
dst_vocab['<sos>'] = len(dst_vocab) + 2
dst_vocab['<eos>'] = len(dst_vocab) + 3

# Обратные словари
src_rev_vocab = {v: k for k, v in src_vocab.items()}
dst_rev_vocab = {v: k for k, v in dst_vocab.items()}

# Размеры словарей
print(f"Размер словаря для src: {len(src_vocab)}")
print(f"Размер словаря для dst: {len(dst_vocab)}")



Размер словаря для src: 180
Размер словаря для dst: 66590


In [45]:
import torch
from torch.nn.utils.rnn import pad_sequence

# Функция для преобразования текста в последовательность индексов
def text_to_sequence(text, vocab):
    return [vocab.get(token, vocab['<unk>']) for token in text]

# Преобразуем все данные в последовательности индексов
train_sequences = [(text_to_sequence(src, src_vocab), text_to_sequence(dst, dst_vocab)) for src, dst in zip(train_src_tokens, train_dst_tokens)]

# Пример преобразования
print(train_sequences[0])  # Пример преобразования в индексы


([0, 1, 2, 3, 4, 5, 6, 7, 3, 8, 9, 10, 2, 11, 12, 10, 13], [0, 1, 2])


In [46]:
# Функция для паддинга последовательностей
def pad_sequences(batch, pad_token):
    src_seqs, dst_seqs = zip(*batch)  # Разделяем src и dst
    src_seqs_padded = pad_sequence([torch.tensor(seq) for seq in src_seqs], padding_value=pad_token)
    dst_seqs_padded = pad_sequence([torch.tensor(seq) for seq in dst_seqs], padding_value=pad_token)
    return src_seqs_padded, dst_seqs_padded

# Пример паддинга
src_pad_token = src_vocab['<pad>']
dst_pad_token = dst_vocab['<pad>']

# Паддинг для обучающего набора
src_padded, dst_padded = pad_sequences(train_sequences, src_pad_token)

print(src_padded.shape)  # Размер после паддинга для src
print(dst_padded.shape)  # Размер после паддинга для dst


torch.Size([321, 300000])
torch.Size([83, 300000])


In [47]:
from torch.utils.data import DataLoader, Dataset

# Кастомный датасет
class TranslationDataset(Dataset):
    def __init__(self, src_data, dst_data):
        self.src_data = src_data
        self.dst_data = dst_data

    def __len__(self):
        return len(self.src_data)

    def __getitem__(self, idx):
        return self.src_data[idx], self.dst_data[idx]

# Создание DataLoader
train_dataset = TranslationDataset(src_padded.t(), dst_padded.t())
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Пример батча
for src_batch, dst_batch in train_loader:
    print(src_batch.shape, dst_batch.shape)
    break  # Пример одного батча


torch.Size([64, 321]) torch.Size([64, 83])


In [48]:
import torch.nn as nn

class Seq2Seq(nn.Module):
    def __init__(self, input_dim, output_dim, emb_dim, hidden_dim, n_layers, dropout):
        super().__init__()
        
        self.encoder = nn.LSTM(emb_dim, hidden_dim, n_layers, dropout=dropout, batch_first=True)
        self.decoder = nn.LSTM(emb_dim, hidden_dim, n_layers, dropout=dropout, batch_first=True)
        self.fc_out = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        
        self.src_embedding = nn.Embedding(input_dim, emb_dim)
        self.dst_embedding = nn.Embedding(output_dim, emb_dim)

    def forward(self, src, dst):
        # Преобразуем входы в эмбеддинги
        src_embedded = self.src_embedding(src)
        dst_embedded = self.dst_embedding(dst)

        # Проходим через энкодер
        encoder_outputs, (hidden, cell) = self.encoder(src_embedded)
        
        # Проходим через декодер
        decoder_outputs, _ = self.decoder(dst_embedded, (hidden, cell))

        # Выход из декодера
        output = self.fc_out(decoder_outputs)
        
        return output


In [49]:
import torch.optim as optim

# Гиперпараметры
input_dim = len(src_vocab)
output_dim = len(dst_vocab)
emb_dim = 256
hidden_dim = 512
n_layers = 2
dropout = 0.5

# Инициализация модели
model = Seq2Seq(input_dim, output_dim, emb_dim, hidden_dim, n_layers, dropout).to(device)

# Оптимизатор и функция потерь
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index=dst_vocab['<pad>'])  # Игнорируем паддинговые токены

# Пример: Проверим модель, чтобы убедиться, что она работает
sample_src = torch.randint(0, input_dim, (32, 10)).to(device)  # Пример входных данных
sample_dst = torch.randint(0, output_dim, (32, 10)).to(device)  # Пример целевых данных
output = model(sample_src, sample_dst)
print(output.shape)  # Проверим размерность выходных данных


torch.Size([32, 10, 66590])


In [ ]:
# Функция обучения
def train(model, train_loader, criterion, optimizer, device):
    model.train()
    epoch_loss = 0

    for src_batch, dst_batch in train_loader:
        src_batch, dst_batch = src_batch.to(device), dst_batch.to(device)

        # Создание целевого индекса (сдвиг на 1, чтобы учить модель предсказывать следующий токен)
        dst_input = dst_batch[:, :-1]
        dst_target = dst_batch[:, 1:]

        # Прямой проход
        optimizer.zero_grad()
        output = model(src_batch, dst_input)

        # Вычисление потерь
        output_dim = output.shape[-1]
        output = output.reshape(-1, output_dim)
        dst_target = dst_target.reshape(-1)
        loss = criterion(output, dst_target)

        # Обратный проход и обновление параметров
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(train_loader)

# Обучение модели
num_epochs = 10
for epoch in range(num_epochs):
    train_loss = train(model, train_loader, criterion, optimizer, device)
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}")


Epoch 1/10, Train Loss: 0.5337
Epoch 2/10, Train Loss: 0.4457
Epoch 3/10, Train Loss: 0.4109
Epoch 4/10, Train Loss: 0.3851
Epoch 5/10, Train Loss: 0.3638
Epoch 6/10, Train Loss: 0.3454
Epoch 7/10, Train Loss: 0.3294
Epoch 8/10, Train Loss: 0.3159


In [18]:
# Токенизация для вымышленного языка (разделение по символам)
def tokenize_fictional(text):
    # Разделяем строку на символы (каждый символ является токеном)
    return list(text)

In [19]:
# Токенизация для английского языка
import spacy
nlp = spacy.load("en_core_web_sm")

def tokenize_english(text):
    # Используем spaCy для лемматизации и удаления стоп-слов и пунктуации
    return [token.lemma_ for token in nlp(text) if not token.is_stop and not token.is_punct]


In [20]:
# Универсальная функция для токенизации в зависимости от языка
def tokenize(text, language="english"):
    if language == "english":
        return tokenize_english(text)
    elif language == "fictional":
        return tokenize_fictional(text)
    else:
        raise ValueError(f"Unsupported language: {language}")

In [21]:
# Токенизация для каждого языка
train_tokens = [
    (tokenize(example['src'], 'fictional'), tokenize(example['dst'], 'english')) 
    for example in train_data
]

val_tokens = [
    (tokenize(example['src'], 'fictional'), tokenize(example['dst'], 'english')) 
    for example in val_data
]

test_tokens = [
    (tokenize(example['src'], 'fictional'), None)  # Поскольку нет перевода в test
    for example in test_data
]

In [22]:
# Пример токенизированных данных
print(train_tokens[0])  # Печать первого токенизированного примера из train_data

(['◄', '▴', '◓', '◠', '▨', ' ', '◨', '▽', '◠', '▦', '◈', '◬', '◓', '▪', '▼', '◬', '▵'], ['Intriguing'])


In [23]:
# Создание словаря для исходного языка (вымышленного языка) и целевого (английского)
src_vocab = Counter([token for pair in train_tokens for token in pair[0]])
dst_vocab = Counter([token for pair in train_tokens for token in pair[1]])

In [24]:
# Сортируем и создаем индексы для словаря
src_vocab = {token: idx + 1 for idx, (token, _) in enumerate(src_vocab.most_common())}
dst_vocab = {token: idx + 1 for idx, (token, _) in enumerate(dst_vocab.most_common())}

In [25]:
# Добавляем специальные токены
src_vocab['<sos>'] = len(src_vocab) + 1  # Start of sentence
src_vocab['<eos>'] = len(src_vocab) + 1  # End of sentence
dst_vocab['<sos>'] = len(dst_vocab) + 1
dst_vocab['<eos>'] = len(dst_vocab) + 1

In [26]:
# Пример словаря
print(f"Размер словаря для исходного языка (вымышленного): {len(src_vocab)}")
print(f"Размер словаря для целевого языка (английского): {len(dst_vocab)}")

# Пример добавления токенов в словарь
print(f"Пример словаря (src): {list(src_vocab.items())[:10]}")  # Печать первых 10 слов
print(f"Пример словаря (dst): {list(dst_vocab.items())[:10]}")  # Печать первых 10 слов

Размер словаря для исходного языка (вымышленного): 178
Размер словаря для целевого языка (английского): 53659
Пример словаря (src): [(' ', 1), ('◠', 2), ('▦', 3), ('◓', 4), ('▱', 5), ('◎', 6), ('◪', 7), ('▴', 8), ('◗', 9), ('◫', 10)]
Пример словаря (dst): [('know', 1), ('go', 2), ('right', 3), ('like', 4), ('think', 5), ('want', 6), ('okay', 7), ('get', 8), ('come', 9), ('tell', 10)]


In [27]:
# Преобразование текста в индексы
def text_to_sequence(text, vocab):
    return [vocab.get(token, vocab['<unk>']) for token in text]


In [29]:
# Добавляем '<unk>' в словарь, если его нет
if '<unk>' not in src_vocab:
    src_vocab['<unk>'] = len(src_vocab)

if '<unk>' not in dst_vocab:
    dst_vocab['<unk>'] = len(dst_vocab)

def text_to_sequence(text, vocab):
    return [vocab.get(token, vocab['<unk>']) for token in text]

# Преобразуем все данные в последовательности индексов
train_sequences = [(text_to_sequence(src, src_vocab), text_to_sequence(dst, dst_vocab)) for src, dst in train_tokens]
val_sequences = [(text_to_sequence(src, src_vocab), text_to_sequence(dst, dst_vocab)) for src, dst in val_tokens]
test_sequences = [(text_to_sequence(src, src_vocab), None) for src, _ in test_tokens]


# Пример преобразования
print(f"Пример индексов для первого примера из train: {train_sequences[0]}")

Пример индексов для первого примера из train: ([54, 8, 4, 2, 12, 1, 21, 19, 2, 3, 11, 16, 4, 18, 30, 16, 13], [24541])


# Подготовка загруженных данных

In [35]:
PAD_TOKEN = "<pad>"

src_vocab[PAD_TOKEN] = len(src_vocab)
dst_vocab[PAD_TOKEN] = len(dst_vocab)


In [38]:
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

# Паддинг последовательностей
def pad_sequences(batch):
    src_seqs, dst_seqs = zip(*batch)  # Разделяем src и dst
    src_seqs_padded = pad_sequence([torch.tensor(seq) for seq in src_seqs], padding_value=src_vocab[PAD_TOKEN])
    dst_seqs_padded = pad_sequence([torch.tensor(seq) for seq in dst_seqs], padding_value=dst_vocab[PAD_TOKEN])
    return src_seqs_padded, dst_seqs_padded

# Создание кастомного collate_fn для DataLoader
def collate_fn(batch):
    src_seqs_padded, dst_seqs_padded = pad_sequences(batch)
    return src_seqs_padded, dst_seqs_padded

# Создание DataLoader с кастомным collate_fn
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)



In [39]:
def mask_padded_sequences(output, target, pad_token_idx):
    # Маскируем паддинг
    mask = target != pad_token_idx
    output = output[mask]
    target = target[mask]
    return output, target

# Модифицируем функцию потерь
def compute_loss(output, target, pad_token_idx):
    output = output.view(-1, output.shape[-1])
    target = target.view(-1)
    output, target = mask_padded_sequences(output, target, pad_token_idx)
    loss = criterion(output, target)
    return loss


# Создание модели Seq2Seq

In [31]:
import torch.nn as nn

class Seq2Seq(nn.Module):
    def __init__(self, input_dim, output_dim, emb_dim, hidden_dim, n_layers, dropout):
        super(Seq2Seq, self).__init__()
        
        # Эмбеддинги
        self.src_embedding = nn.Embedding(input_dim, emb_dim)
        self.dst_embedding = nn.Embedding(output_dim, emb_dim)
        
        # LSTM энкодер
        self.encoder = nn.LSTM(emb_dim, hidden_dim, n_layers, dropout=dropout)
        
        # LSTM декодер
        self.decoder = nn.LSTM(emb_dim, hidden_dim, n_layers, dropout=dropout)
        
        # Линейный слой для выхода
        self.fc_out = nn.Linear(hidden_dim, output_dim)
        
        # Дропаут
        self.dropout = nn.Dropout(dropout)

    def forward(self, src, dst):
        # Эмбеддим входные последовательности
        src_emb = self.src_embedding(src)
        dst_emb = self.dst_embedding(dst)

        # Процессинг с помощью энкодера
        encoder_output, (hidden, cell) = self.encoder(src_emb)

        # Декодируем выход энкодера
        decoder_output, _ = self.decoder(dst_emb, (hidden, cell))
        
        # Прогнозируем выход
        output = self.fc_out(decoder_output)
        
        return output


# Определение потерь и оптимизатора

In [32]:
# Параметры модели
input_dim = len(src_vocab)
output_dim = len(dst_vocab)
emb_dim = 256
hidden_dim = 512
n_layers = 2
dropout = 0.5

# Инициализация модели
model = Seq2Seq(input_dim, output_dim, emb_dim, hidden_dim, n_layers, dropout)

# Функция потерь
criterion = nn.CrossEntropyLoss()

# Оптимизатор
optimizer = torch.optim.Adam(model.parameters())


# Обучение модели

In [33]:
def train(model, train_loader, criterion, optimizer, device):
    model.train()
    epoch_loss = 0
    for src, dst in train_loader:
        src = src.to(device)
        dst = dst.to(device)

        # Обнуляем градиенты
        optimizer.zero_grad()

        # Прогоняем через модель
        output = model(src, dst)

        # Рассчитываем потери
        output_dim = output.shape[-1]
        output = output.view(-1, output_dim)
        dst = dst.view(-1)

        loss = criterion(output, dst)
        loss.backward()

        # Обновляем параметры
        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(train_loader)


In [34]:
# Переносим модель на устройство (GPU/CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Обучаем модель
num_epochs = 10

for epoch in range(num_epochs):
    train_loss = train(model, train_loader, criterion, optimizer, device)
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}")

RuntimeError: stack expects each tensor to be equal size, but got [30] at entry 0 and [56] at entry 1